Neural netowrk funcs

Layers: an Integer value representing the total number of hidden layers in the network
(input and output layers are extra)

● Nodes: an integer array of size [0,..,Layers+1] containing the dimensions of the neural
network. Nodes[0] shall represent the input size (typically, 50), Nodes[Layers+1]

shall represent the number of output nodes (typically, 1). All other values Nodes[i]
represent the number of nodes in hidden layer i.
● NNodes: a possible alternative to the Nodes parameter for situations where you want
each hidden layer of the neural network to be of the same size. In this case, the size of
the output layer is assumed to be 1, and the size of the input layer can be inferred from
the dataset.
● Activations: an array of size [0,..,Layers+1] (for the sake of compatibility) in which
Activations[0] and Activations[Layers+1] are not used, while all other
Activations[i] values are labels indicating the activation function used in layer i.
This allows you to build neural networks with different activation functions in each layer.
● ActivationFn: a possible alternative to Activations when all hidden layers of your neural
network use the same activation function.

params: layers: # of hidden layers, 
	nodes: if int: number of nodes for each hidden layer (when creating hidden layer, add 1 extraa node for bias term)
	       if arr: number elems in arr should be same as layers, each num will be number of nodes for the hidden layer
	activation: apply to all hidden layers
	learning rate:
	batch size:


when initializing, need to generate random weights for all the layers

x: num_obs x num_features

w1+b: num_features(x) + 1 x width + 1 (of h1)

h1 = num_obs x width+1(of h2)

w2+b: width+1(of h1) x width+1 (of h2)
*****

h2 = num_obs x width+1(of h2) (of h2)

w3+b = width+1(of h2) x 1(we only want it output a single value per observation)


y = num_obs x 1

In [2]:
import numpy as np
import pandas as pd

In [185]:
class NeuralNetwork:
    
    def __init__(self, layers, nodes, activation, loss, learning_rate, batch_size):
        
        # number of hidden layers we are creating
        self.num_layers = layers
        
        # number of nodes at each layer
        self.nodes = self._create_nodes(nodes,layers)
        
        # activation function
        self.sigma = activation
                
        # loss function
        self.loss = loss
        
        # learning rate for stochastic gradient descent
        self.lr = learning_rate
        
        # batch size for stochastic gradient descent
        self.batch_size = batch_size

        # after fit, will contain a dictionary of weights for the layers
        self.weight_dict = None
        
        # store values for backwards propagation
        self.stored = {}

        
    def _create_nodes(self, nodes, layers):
    
        if isinstance(nodes, int):
            nodes = nodes+1
            return [nodes]*(layers)
        
        else:
            return [i+1 for i in nodes]
        
    def _create_weights(self, num_features):
        
        # for 3 hidden layers
        
        #0 between input layer and hidden layer 1: 
        #1 hidden layer 1 to hidden layer 2: width x width
        #2 hidden layer 2 to hidden layer 3: width x width
        #3 hidden layer 3 to y :width x 1
        
        #so self.layers[0] will always be numfeatures x width(of first hidden layer)
        # self.layers[num_layers] = widthx1
        
        self.weight_dict = {
            0: np.random.rand(num_features, self.nodes[0])
        }
        
        for i in range(1,self.num_layers):
            self.weight_dict[i] = np.random.rand(self.nodes[i-1], self.nodes[i])
            
        self.weight_dict[self.num_layers] = np.random.rand(self.nodes[self.num_layers-1], 1)
        
        
    def _forward_propagation(self, X, y):
        # need to figure out what weights to save and how to backprop
        
        sigma_h_x = X
        
        for i in range(0,self.num_layers):
            h_x = sigma_h_x @ self.weight_dict[i]
            sigma_h_x = self.sigma(h_x) 
            self.stored[i] = [h_x, sigma_h_x]
        
        z_sigma_h_x = sigma_h_x @ self.weight_dict[self.num_layers]
        self.stored[self.num_layers] = [z_sigma_h_x]
        
        return np.sum(self.loss(z_sigma_h_x, y))/self.batch_size #summation of num_obs x 1 array divide by batchsize
        """
        2 hidden layer example
        weight 0: [h1(x), sigma(h1(x))] #X to h1
        weight 1: [h2(sigma(h1(x))), sigma(h2(sigma(h1(x))))] #h1 to h2 [bxn], [bxn]
        weight 2: [z(sigma(h2(sigma(h1(x)))))] #h2 to y [nx1]
        """
    def _backward_propagation(self, X,y):

        expected = self.stored[self.num_layers][0] #z(sigma(h2(sigma(h1(x)))))
        
         # dL/dz(z(sigma(h2(sigma(h1(x))))))
        J = self.loss(expected, y, derivative = True)
        old_weights = self.weight_dict[self.num_layers] 
        
        #updating the weights
        #dz/dw2(sigma(h2(sigma(h1(x)))))
        self.weight_dict[num_layers] = self.weight_dict[num_layers] - self.lr*(self.stored[num_layers-1][1].T @ J)
        #dz/dsigma(sigma(h2(sigma(h1(x)))))
        J = J @ old_weights.T 
        
        ################################################################################
        
        #dsigma/dh2(h2(sigma(h1(x))))) # i dont understand
        J = self.sigma(J, derivative = True)
        # J = self.sigma(self.stored[num_layers-1][0], derivative)
        
        #updating the weights
        #dh2/w1
        #dh2/sigma(sigma(h2(sigma(h1(x)))))
        old_weights = self.weight_dict[self.num_layers-1] 
        self.weight_dict[num_layers-1] = self.weight_dict[num_layers-1] -self.lr*(self.stored[num_layers-2][1].T @ J)
        
        #updating the weights
        #dh2/dsigma(sigma(h2(sigma(h1(x)))))
        J = J @ old_weights.T
        
        # network: X -w0-sigma-> h1 -w1-sigma-> h2 -w2-> y
        
        #1. J = loss_deriv(z(sigma(h2(sigma(h1(x))))))
        #2. Update weight w2 = w2 - learning_rate*(sigma(h2(sigma(h1(x)))) @ J)
        
        #3  Dense layer derivative on z AKAJ =J @ oldweights.T
        #4. Activation Layer derivative of J: J = J* sigmad_deriv(h2(sigma(h1(x))))
        #5. Update weight w1 = w1 - learning_rate*(sigma(h1(x))) @ J)
        
        
        
        
        
        # dense layer derivative on z AKA J @ oldweights.T
         #4. Activation Layer derivative of J: J = J* sigmad_deriv((h1(x))))
            
            #5. Update weight w0 = w0 - learning_rate*(x)) @ J)
        
        
        
        for i in range(self.num_layers,0,-1):
            
            
            
            
            
            
            
            
            
            
            
            old_weights = self.weight_dict[i]
            self.weight_dict[i] = self.weight_dict[i] -self.lr*(self.stored[i-1][1].T @ J) #dz/dw | dh/dw
            print(self.stored[i-1][1].T.shape)
            print(J.shape)
            J =  self.stored[i-1][1].T @ J #dz/dsigma  | dh/dsigma
            print("\n")
            print(J.shape)
            print(self.sigma(self.stored[i-1][0], derivative = True).shape)
            J =  J @ self.sigma(self.stored[i-1][0], derivative = True) #dsigma/dh
            

        #final weight update
        self.weight_dict[0] = X.T @ J #dh1/dw0(x)
        
        
    
    
  dZ1 = np.multiply(dA1, A1 * (1 - A1)) 
    def _stochastic_gradient_descent(self, X, y, num_iterations, print_iter):
        # how to integrate forward and back vias sgd
        
        avg_err_arr = [np.nan]*num_iterations
        
        indices = np.random.choice(X.shape[0], self.batch_size, replace=False)
        x_batch = X[indices]
        y_batch =y[indices]
        
        avg_err_arr[0] = self._forward_propagation(batch,y_batch)
        
        for i in range(1,num_iterations):

            self._backward_propagation(x_batch,y_batch)
            indices = np.random.choice(X.shape[0], self.batch_size, replace=False)
            x_batch = X[indices]
            y_batch =y[indices]
        
            avg_err_arr[i] = self._forward_propagation(batch,y_batch)
            
        
        return avg_err_arr
        
    
    def fit(self, X,y, num_iterations = 1000, print_iter = False):
        
        # add intercept to X
        X = np.hstack([np.ones(len(x))[:, np.newaxis], x])  
        y = np.array([y]).T # makes it easier to calculate loss
        
        # generate the random weights for every layer in neural network
        self._create_weights(X.shape[1])
        
        self._stochastic_gradient_descent(X,y,num_iterations, print_iter)
        
        #self._forward_propagation(X,y)
        
        pass
    
    # forward propagation without saved vals for speed
    def predict(self, X):
        sigma_h_x = X
        
        for i in range(0,self.num_layers):
            h_x = sigma_h_x @ self.weight_dict[i]
            sigma_h_x = self.sigma(h_x) 
        
        z_sigma_h_x = sigma_h_x @ self.weight_dict[num_layers]
        return z_sigma_h_x
        

In [193]:
def L2_loss(x,y, derivative = False):
    # assumes y is nx1 vector (n rows, 1 col)
    if derivative:
        return x-y
    else:
        return 0.5*((x-y)**2)
        
def ReLU(X, derivative = False):
    if derivative:
        # faster than np.where
        np.greater(X, 0).astype(int)
    return np.maximum(X,0)

In [201]:
layers = 3
nodes = 1
activation = ReLU
loss = L2_loss
learning_rate = 0.001
batch_size = 2


test_nn = NeuralNetwork(layers, nodes, activation, loss, learning_rate, batch_size)

In [202]:
test_nn.nodes

[2, 2, 2]

In [203]:
X = np.array([
    [1,2,1],
    [3,5,5],
    [3,1,2]
    
])

y = np.array([[5,14,7]]).T


test_nn._create_weights(X.shape[1])

In [216]:
test_nn.weight_dict[3]

array([[0.2159043 ],
       [0.96872786]])

In [208]:
test_nn._forward_propagation( X, y)

12.099576927450684

In [210]:
test_nn.stored[0][0]

array([[1.44527635, 2.01644947],
       [4.63699299, 6.05289848],
       [2.20976458, 2.6436362 ]])

In [211]:
test_nn.stored[0][1]

array([[1.44527635, 2.01644947],
       [4.63699299, 6.05289848],
       [2.20976458, 2.6436362 ]])

In [214]:
test_nn.stored[2][0]

array([[0.869055  , 2.56440005],
       [2.70716886, 7.95124233],
       [1.24322286, 3.62941617]])

In [205]:
test_nn._backward_propagation(X,y)
test_nn._forward_propagation(X,y)

(2, 3)
(3, 1)


(2, 1)
(3, 2)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 1)

In [191]:
"""

#         Jw1 = self.stored[self.num_layers-2][1].T @ J #dh2/dw1(sigma(h1(x)))
#         J =  J @ self.sigma(self.stored[self.num_layers-2][1], derivative = True) #dh2/dsigma(sigma(h1(x)))
#         J =  J @ self.stored[self.num_layers-2][0].T  #dsigma/dh1(h1(x))
        

"""

'\n\n#         Jw1 = self.stored[self.num_layers-2][1].T @ J #dh2/dw1(sigma(h1(x)))\n#         J =  J @ self.sigma(self.stored[self.num_layers-2][1], derivative = True) #dh2/dsigma(sigma(h1(x)))\n#         J =  J @ self.stored[self.num_layers-2][0].T  #dsigma/dh1(h1(x))\n        \n\n'